케라스(Keras)로 RNN 구현하기

- RNN 층을 추가하는 코드

model.add(SimpleRNN(hidden_size)) # 가장 간단한 형태

- 추가 인자 사용 

model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))


model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N))

 - hidden_size = 은닉 상태의 크기( 128, 256, 512, 1024 )
 - timesteps = 입력 시퀀스의 길이(input_length), 시점의 수
 - input_dim = 입력의 크기

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN

In [3]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
# 출력값이 (batch_size, output_dim) 크기의 2D 텐서
# hidden_size의 값: 3
model.summary()
# batch_size를 현 단계에서는 알 수 없으므로 (None, 3)이 됨

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))

In [4]:
# batch size 정의 (batcj size = 8)
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [5]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()
# return_sequences=True
# 출력: (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


파이썬으로 RNN 구현

In [6]:
import numpy as np

timesteps = 10 
# 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 
# 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 
# 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) 
# 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) 
# 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

In [7]:
# 초기 은닉 상태 출력 
print(hidden_state_t) 
# 8의 크기를 가지는 은닉 상태. 
# 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.
# 8의 차원을 가지는 0의 값으로 구성된 벡터가 출력

[0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# 가중치, 편향 정의 
Wx = np.random.random((hidden_size, input_dim))  
# (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) 
# (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) 
# (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

In [10]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [11]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: 
  # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) 
  # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t))
  # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) 
  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t
  # 출력 결과가 은닉 상태로 다음 time step이 됨 

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)


In [13]:
total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.
total_hidden_states

array([[0.89423164, 0.93383248, 0.86257327, 0.978129  , 0.81613124,
        0.87143064, 0.97861428, 0.68015318],
       [0.99980352, 0.99999008, 0.99921648, 0.99999589, 0.99960354,
        0.99931738, 0.99998951, 0.9999298 ],
       [0.99995039, 0.99999798, 0.9996289 , 0.99999929, 0.99993692,
        0.99968139, 0.9999973 , 0.99999257],
       [0.99991027, 0.99999828, 0.9994628 , 0.99999824, 0.99987473,
        0.99959254, 0.99999568, 0.99998977],
       [0.99990032, 0.99999177, 0.99888059, 0.99999594, 0.99979914,
        0.9992103 , 0.99999034, 0.99998044],
       [0.99995982, 0.99999772, 0.99961848, 0.99999912, 0.99992902,
        0.99970131, 0.99999818, 0.9999921 ],
       [0.99986636, 0.99998807, 0.99832379, 0.99999179, 0.99969656,
        0.99892224, 0.99998524, 0.99997408],
       [0.99992257, 0.99999633, 0.99927601, 0.99999727, 0.99984017,
        0.99949591, 0.99999558, 0.99998611],
       [0.99992489, 0.9999952 , 0.99922932, 0.99999765, 0.99987316,
        0.99943536, 0.999994

깊은 순환 신경망(Deep Recurrent Neural Network)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

In [15]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))